***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-374e  GCE       3                                             RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
from operator import add


import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Mar  9 15:16 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [6]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'nettaya-315443382' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name.endswith('.parquet'):        
        paths.append(full_path+b.name)

In [8]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [9]:
from inverted_index_gcp import *

In [10]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("id", "title").rdd

In [ ]:
# Map the RDD into a dictionary
doc_title_dict = doc_text_pairs.map(lambda x: (x[0], x[1])).collectAsMap()


# # Serialize the dictionary to bytes
# bytes_obj = BytesIO()
# pickle.dump(doc_title_dict, bytes_obj)
# bytes_obj.seek(0)  # Rewind the BytesIO object to the beginning
# index_src = "index_id_title.pkl"
# index_dst = f'gs://{bucket_name}/{index_src}'
# # Get the bucket object and specify the blob (file) path
# bucket = client.get_bucket(bucket_name)
# blob = bucket.blob(index_dst)

# # Upload the serialized bytes to GCS
# blob.upload_from_file(bytes_obj, content_type='application/octet-stream')

# print(f'Successfully saved {index_src} to gs://{bucket_name}/{index_dst}')


In [ ]:
# from itertools import islice

# # Assuming doc_title_dict is your dictionary
# first_20_items = dict(islice(doc_title_dict.items(), 20))
# print(first_20_items)


In [ ]:
from io import BytesIO



# The source (local filename) and destination (GCS path) for the pickle file
index_src = "index_id_title.pkl"  # Local filename
index_dst = f'gs://{bucket_name}/{index_src}'  # GCS path
!gsutil cp index_id_title.pkl gs://nettaya-315443382/index_id_title.pkl

